<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250316%5D%20Thesis%20hybrid%20RF-LSTM%20with%20FGSM%20and%20PGD%20but%20failed%20DeepFool%20on%20WSN-BFSF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Imports various Python libraries necessary
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils import np_utils
#from keras.utils import np_utils
#from tensorflow.keras import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification

In [2]:
# Load dataset
dataset = pd.read_csv("dataset.csv")

# Preprocessing: Handle missing values (if any)
dataset = dataset.dropna() # Remove missing values

In [3]:
dataset

,Event,Time,S_Node,Node_id,Rest_Energy,Trace_Level,Mac_Type_Pckt,Source_IP_Port,Des_IP_Port,Packet_Size,TTL,Hop_Count,Broadcast_ID,Dest_Node_Num,Dest_Seq_Num,Src_Node_ID,Src_Seq_Num,Class
0,1,0.100000,79,79,600.000000,5,0,79.255,1.255,48,30,1,1,100,0,79,4,0
1,2,0.100963,78,78,599.979723,5,800,79.255,1.255,48,30,1,1,100,0,79,4,0
2,2,0.100963,76,76,599.979722,5,800,79.255,1.255,48,30,1,1,100,0,79,4,0
3,2,0.100964,75,75,599.979722,5,800,79.255,1.255,48,30,1,1,100,0,79,4,0
4,2,0.100964,118,118,599.979722,5,800,79.255,1.255,48,30,1,1,100,0,79,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312101,4,656.526536,140,140,465.332078,5,800,140.255,1.255,48,30,1,31,100,16,140,68,1
312102,4,657.209643,140,140,465.191806,5,800,140.255,1.255,48,30,1,33,100,16,140,72,1
312103,4,657.891729,140,140,465.051737,5,800,140.255,1.255,48,30,1,35,100,16,140,76,1
312104,4,658.574656,140,140,464.911501,5,800,140.255,1.255,48,30,1,37,100,16,140,80,1


In [4]:
# Encode non-numeric columns (if any)
for column in dataset.columns:
    if dataset[column].dtype == 'object':  # Identify categorical columns
        dataset[column] = LabelEncoder().fit_transform(dataset[column])

# Separate features (X) and target (y)
X = dataset.drop(['Class'], axis=1)  # Assuming 'Class' is the target column
y = dataset['Class']

print("Original Class Distribution:", np.bincount(y))

Original Class Distribution: [262851  11766   7645  29844]


In [5]:
from sklearn.preprocessing import LabelEncoder, StandardScaler # Import StandardScaler here
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [6]:
!pip install -U imbalanced-learn

In [7]:
from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the data
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Verify the balanced dataset distribution
print("Balanced Class Distribution:", np.bincount(y))

Balanced Class Distribution: [262851 262851 262851 262851]


In [8]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)

In [9]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, TimeDistributed
from tensorflow.keras.regularizers import l2
import numpy as np

In [10]:
!pip install adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.2 MB/s eta 0:00:00


In [11]:
# Train RF model for probability extraction
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Use RF probabilities instead of direct predictions
train_rf_probs = rf_model.predict_proba(X_train)
test_rf_probs = rf_model.predict_proba(X_test)

# Reshape for LSTM input
train_rf_features = np.reshape(train_rf_probs, (train_rf_probs.shape[0], 1, train_rf_probs.shape[1]))
test_rf_features = np.reshape(test_rf_probs, (test_rf_probs.shape[0], 1, test_rf_probs.shape[1]))

In [12]:
from tensorflow.keras.utils import to_categorical
# One-hot encode target
y_train_enc = to_categorical(y_train)
y_test_enc = to_categorical(y_test)

# LSTM model
lstm_model = Sequential([
    LSTM(64, input_shape=(train_rf_features.shape[1], train_rf_features.shape[2]), return_sequences=True, kernel_regularizer=l2(0.0001)),
    Dropout(0.2),
    LSTM(32, return_sequences=True, kernel_regularizer=l2(0.0001)),
    Dropout(0.2),
    LSTM(16, kernel_regularizer=l2(0.0001)),
    Dropout(0.2),
    Dense(y_train_enc.shape[1], activation='softmax')
])

lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training LSTM
lstm_model.fit(train_rf_features, y_train_enc, epochs=30, batch_size=64, validation_split=0.2, verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
10515/10515 ━━━━━━━━━━━━━━━━━━━━ 76s 7ms/step - accuracy: 0.9823 - loss: 0.1141 - val_accuracy: 1.0000 - val_loss: 0.0018
Epoch 2/30
10515/10515 ━━━━━━━━━━━━━━━━━━━━ 72s 7ms/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 1.0000 - val_loss: 9.8109e-04
Epoch 3/30
10515/10515 ━━━━━━━━━━━━━━━━━━━━ 71s 7ms/step - accuracy: 1.0000 - loss: 9.8909e-04 - val_accuracy: 1.0000 - val_loss: 7.4099e-04
Epoch 4/30
10515/10515 ━━━━━━━━━━━━━━━━━━━━ 72s 7ms/step - accuracy: 1.0000 - loss: 8.2427e-04 - val_accuracy: 1.0000 - val_loss: 6.2374e-04
Epoch 5/30
10515/10515 ━━━━━━━━━━━━━━━━━━━━ 72s 7ms/step - accuracy: 1.0000 - loss: 7.4835e-04 - val_accuracy: 1.0000 - val_loss: 6.2836e-04
Epoch 6/30
10515/10515 ━━━━━━━━━━━━━━━━━━━━ 72s 7ms/step - accuracy: 1.0000 - loss: 6.4229e-04 - val_accuracy: 1.0000 - val_loss: 5.3744e-04
Epoch 7/30
10515/10515 ━━━━━━━━━━━━━━━━━━━━ 72s 7ms/step - accuracy: 1.0000 - loss: 6.0162e-04 - val_accuracy: 1.0000 - val_loss: 5.0691e-04
Epoch 8/30
10515/10515 ━━

In [13]:
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent, CarliniL2Method
from sklearn.metrics import accuracy_score
from tensorflow.keras.losses import CategoricalCrossentropy

# ART Classifier for adversarial robustness
classifier = TensorFlowV2Classifier(
    model=lstm_model,
    nb_classes=y_train_enc.shape[1],
    input_shape=(1, train_rf_features.shape[2]),
    loss_object=CategoricalCrossentropy()
)

# FGSM Attack
fgsm = FastGradientMethod(estimator=classifier, eps=0.2)
X_test_adv_fgsm = fgsm.generate(test_rf_features)
y_pred_fgsm = np.argmax(classifier.predict(X_test_adv_fgsm), axis=1)
accuracy_fgsm = accuracy_score(y_test, y_pred_fgsm)
print(f"Accuracy under FGSM attack: {accuracy_fgsm:.10f}")

Accuracy under FGSM attack: 0.9980549836


In [14]:
# PGD Attack
pgd = ProjectedGradientDescent(estimator=classifier, eps=0.2, max_iter=20, batch_size=32)
X_test_adv_pgd = pgd.generate(test_rf_features)
y_pred_pgd = np.argmax(classifier.predict(X_test_adv_pgd), axis=1)
accuracy_pgd = accuracy_score(y_test, y_pred_pgd)
print(f"Accuracy under PGD attack: {accuracy_pgd:.10f}")

PGD - Batches: 0it [00:00, ?it/s]

Accuracy under PGD attack: 0.9980549836


In [17]:
pip install adversarial-robustness-toolbox --upgrade

In [18]:
from art.attacks.evasion import DeepFool  # Ensure this line is present

In [19]:
# ART classifier setup
classifier = TensorFlowV2Classifier(
    model=lstm_model,
    nb_classes=y_train_enc.shape[1],
    input_shape=(train_rf_features.shape[1], train_rf_features.shape[2]),
    loss_object=CategoricalCrossentropy()
)

# DeepFool Attack Generation
deepfool_attack = DeepFool(classifier=classifier, max_iter=20, batch_size=32)
X_test_adv_deepfool = deepfool_attack.generate(train_rf_features)

# Evaluate accuracy under DeepFool attack
y_pred_deepfool = np.argmax(classifier.predict(X_test_adv_deepfool), axis=1)
accuracy_deepfool = accuracy_score(y_test, y_pred_deepfool)
print(f"Accuracy under DeepFool attack: {accuracy_deepfool:.10f}")

DeepFool:   0%|          | 0/26286 [00:00<?, ?it/s]

ValueError: Found input variables with inconsistent numbers of samples: [210281, 841123]

In [20]:
from art.attacks.evasion import DeepFool  # Ensure this line is present

# ART classifier setup
classifier = TensorFlowV2Classifier(
    model=lstm_model,
    nb_classes=y_train_enc.shape[1],
    input_shape=(train_rf_features.shape[1], train_rf_features.shape[2]),
    loss_object=CategoricalCrossentropy()
)

# Select a smaller subset for faster adversarial example generation (optional, but recommended)
subset_size = min(500, train_rf_features.shape[0])  # use at most 500 samples
train_rf_features_subset = train_rf_features[:subset_size]
y_test_subset = y_test[:subset_size]

# DeepFool Attack Generation (lighter and faster)
deepfool_attack = DeepFool(
    classifier=classifier,
    max_iter=20,               # Significantly reduced from 20
    epsilon=1e-4,             # Increased epsilon for faster convergence
    batch_size=8             # Increased batch size
)

X_test_adv_deepfool = deepfool_attack.generate(train_rf_features_subset)

# Evaluate accuracy under DeepFool attack
y_pred_deepfool = np.argmax(classifier.predict(X_test_adv_deepfool), axis=1)
accuracy_deepfool = accuracy_score(y_test_subset, y_pred_deepfool)
print(f"Accuracy under DeepFool attack: {accuracy_deepfool:.10f}")

DeepFool:   0%|          | 0/63 [00:00<?, ?it/s]

Accuracy under DeepFool attack: 0.2140000000
